In [ ]:
%load_ext autoreload
%autoreload 2

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# some imports from my project library
from turboflow.dataloaders import load_turbo2D_simple_numpy
from turboflow.utils import phy_utils as phy
from turboflow.utils import dsp_utils as dsp
from turboflow.utils import torch_utils as tch
from turboflow.utils import viz_utils as viz
from turboflow import evaluation as evl


# import torch lib
import torch
# import pl torch libs
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from turboflow.dataloaders import Turbo2DDataModule

In [8]:
dm = Turbo2DDataModule(train_ds=4, val_ds=4,test_ds=1,img_idx=42)
dm.prepare_data()

Y shape (64, 64, 2)
Y min, max: -2.365612 2.941536
after normalization, Y min, max: -0.8042097733972999 1.0
Y shape (64, 64, 2)
Y min, max: -2.365612 2.941536
after normalization, Y min, max: -0.8042097733972999 1.0
Y shape (256, 256, 2)
Y min, max: -2.382122 2.999472
after normalization, Y min, max: -0.7941804424245333 1.0


In [9]:
for batch in dm.train_dataloader():
    X, y = batch
    print(X.shape)
    print(y.shape)

for batch in dm.val_dataloader():
    X, y = batch
    print(X.shape)
    print(y.shape)
    print(X[:2,:])
    print(X[2,1]-X[2,0])
    
for batch in dm.test_dataloader():
    X, y = batch
    print(X.shape)
    print(y.shape)
    print(X[:2,:])
print('%1.8f' % (X[2,1]-X[2,0]))


torch.Size([4096, 2])
torch.Size([4096, 2])
torch.Size([4096, 2])
torch.Size([4096, 2])
tensor([[0.0000, 0.0000],
        [0.0000, 0.0157]])
tensor(0.0314)
torch.Size([65536, 2])
torch.Size([65536, 2])
tensor([[0.0000, 0.0000],
        [0.0000, 0.0039]])
0.00784314


In [39]:
from turboflow.models.phyrff_hard import plDivFreeRFFNet

# model (RFF + MLP)
do_rff = True
fft_scale = 10
fft_nfeat = 256

nin = 2  # = x and y coordinates
nout = 1 # = in the mlp we predict the potential, ux and uy components followed
mlp_layers = [nin] + [256]*3 + [nout]
last_activation_fun = torch.nn.Tanh()

delta_x = 0.00784314 # hardcoded
n_centers = 5
n_increments = 5

lam_pde = 1
lam_reg = 1
lam_sfn = 1

model = plDivFreeRFFNet('DivFreeNet_RFF', 
                        mlp_layers, last_activation_fun,
                        do_rff, fft_nfeat, fft_scale,
                        lam_pde, lam_reg, lam_sfn,
                        smallest_increment=delta_x, n_centers=n_centers, n_increments=n_increments,
                       )

In [44]:
early_stop_callback = EarlyStopping(monitor='valid_loss')


trainer = pl.Trainer(gpus=1, 
                     check_val_every_n_epoch=200, 
                     max_epochs=5000, 
                     callbacks=[early_stop_callback])

trainer.fit(model, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | rff  | Fourier | 512   
1 | mlp  | MLP     | 263 K 
2 | div  | DivFree | 0     
---------------------------------
263 K     Trainable params
512       Non-trainable params
263 K     Total params
1.055     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

5 10 0.00784314 32 cuda:0


RuntimeError: CUDA out of memory. Tried to allocate 400.00 MiB (GPU 0; 15.75 GiB total capacity; 13.81 GiB already allocated; 212.56 MiB free; 14.01 GiB reserved in total by PyTorch)